<a href="https://colab.research.google.com/github/JannisWolf/evaluating-edge-accelerator/blob/main/model_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim

import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls '/content/gdrive/My Drive/data/autoencoder'

net-state-3072-scaled2-standard


In [ ]:
inputSize = 3*1024

In [ ]:
# auto encoder class

class GrindNet(nn.Module):
  def __init__(self, **kwargs):
    super().__init__()
    sizes = [inputSize, 256]
    self.layers = nn.ModuleList()

    for size1, size2 in zip(sizes, sizes[1:]):
      self.layers.append(nn.Linear(in_features=size1, out_features=size2))
    for size1, size2 in zip(reversed(sizes), list(reversed(sizes))[1:]):
      self.layers.append(nn.Linear(in_features=size1, out_features=size2))
  
  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
      x = functional.leaky_relu(x)
    return x

In [ ]:
model = GrindNet()
path = F"/content/gdrive/My Drive/data/autoencoder/"
model_name = "net-state-3072-scaled2-standard"

In [ ]:
model.load_state_dict(torch.load(path+model_name))
model.eval()

GrindNet(
  (layers): ModuleList(
    (0): Linear(in_features=3072, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=3072, bias=True)
  )
)

In [ ]:
with torch.no_grad():
     torch.onnx.export(model,
     torch.randn(*(inputSize,)),
     path + "/model.onnx",
     export_params=True,
     opset_version=10,
     do_constant_folding=True,
     input_names=['input'],
     output_names=['output'])

In [ ]:
!pip install onnx onnx2keras pytorch2keras


  Created wheel for pytorch2keras: filename=pytorch2keras-0.2.4-cp37-none-any.whl size=29677 sha256=4fc719b9d6a7335cde4b71faf4b1346aaf733058bb269cf119512e633da6d23d
  Stored in directory: /root/.cache/pip/wheels/36/8b/2e/e2b6ae7a78ad4661e6156700bf96816309013f89f7a21f82d7
Successfully built pytorch2keras


In [ ]:
import onnx
from onnx2keras import onnx_to_keras
from pytorch2keras.converter import pytorch_to_keras


ImportError: ignored

In [ ]:
# Load ONNX model
onnx_model = onnx.load(path + 'model.onnx')

# Call the converter (input - is the main model input name, can be different for your model)
k_model = onnx_to_keras(onnx_model, ['input'])

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight layers.0.bias with shape (256,).
DEBUG:onnx2keras:Found weight layers.1.bias with shape (3072,).
DEBUG:onnx2keras:Found weight 13 with shape (3072, 256).
DEBUG:onnx2keras:Found weight 14 with shape (256, 3072).
DEBUG:onnx2keras:Found input input with shape []
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: MatMul
DEBUG:onnx2keras:node_name: 6
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input).
DEBUG:onnx2keras:Check input 1 (name 13).
DEBUG:onnx2keras:The input not found in layers

ValueError: ignored

In [ ]:
!onnx-tf convert -i "dummy_model.onnx" -o  'dummy_model_tensorflow'

/bin/bash: onnx-tf: command not found


Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package onnx-tf
